In [1]:
# Load libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics  

In [2]:
# Load training data
train_data = pd.read_csv('train.csv')
len(train_data)

36992

In [3]:
train_data.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [4]:
# Load testing data
test_data = pd.read_csv('test.csv')
len(test_data)

19919

In [5]:
test_data.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,xxxxxxxx,...,12,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,xxxxxxxx,...,11,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,xxxxxxxx,...,18,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,...,-999,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,...,6,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [6]:
# Get the number of missing data points per column
missing_values_count_train = train_data.isnull().sum()
print(missing_values_count_train)

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64


In [7]:
# Get the number of missing data points per column
missing_values_count_test = test_data.isnull().sum()
print(missing_values_count_test)

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 2948
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            159
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                1963
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
dtype: int64


In [8]:
# Dropping all rows with missing data
#train_modified = train_data.dropna()
train_imputed = train_data.fillna(method='bfill', axis=0).fillna(0)
train_imputed.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,...,306.34,12838.38,10.0,500.69,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [9]:
# Dropping all rows with missing data
test_imputed = test_data.fillna(method='bfill', axis=0).fillna(0)
test_imputed.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,xxxxxxxx,...,12,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,xxxxxxxx,...,11,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,xxxxxxxx,...,18,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,...,-999,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,...,6,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [10]:
# Plot statistics of Age
print(train_imputed['age'].describe())

count    36992.000000
mean        37.118161
std         15.867412
min         10.000000
25%         23.000000
50%         37.000000
75%         51.000000
max         64.000000
Name: age, dtype: float64


In [11]:
# Plot statistics of Average time spent
print(train_imputed['avg_time_spent'].describe())

count    36992.000000
mean       243.472334
std        398.289149
min      -2814.109110
25%         60.102500
50%        161.765000
75%        356.515000
max       3235.578521
Name: avg_time_spent, dtype: float64


In [12]:
# Plot statistics of Average transaction value
print(train_imputed['avg_transaction_value'].describe())

count    36992.000000
mean     29271.194003
std      19444.806226
min        800.460000
25%      14177.540000
50%      27554.485000
75%      40855.110000
max      99914.050000
Name: avg_transaction_value, dtype: float64


In [13]:
# Plot statistics of Average frequency login days value
train_imputed['avg_frequency_login_days'] = train_imputed['avg_frequency_login_days']
print(train_imputed['avg_frequency_login_days'].describe())

count     36992
unique     1654
top       Error
freq       3522
Name: avg_frequency_login_days, dtype: object


In [14]:
# Plot statistics of wallet points
print(train_imputed['points_in_wallet'].describe())

count    36992.000000
mean       686.676246
std        194.492744
min       -760.661236
25%        616.320000
50%        697.795000
75%        763.740000
max       2069.069761
Name: points_in_wallet, dtype: float64


In [15]:
features_num = ['age', 'avg_time_spent', 'avg_transaction_value', 'points_in_wallet'] #'avg_frequency_login_days'
features_cat = ['gender', 'region_category', 'membership_category', 'complaint_status', 'feedback']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

y = train_imputed.pop('churn_risk_score')
#train_imputed = train_imputed.iloc[:,[1,2,3,10,12]]
X = preprocessor.fit_transform(train_imputed)

#test_imputed = test_imputed.iloc[:,[1,2,3,10,12]]
test_X = preprocessor.fit_transform(test_imputed)

train_imputed.columns

Index(['customer_id', 'Name', 'age', 'gender', 'security_no',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'referral_id', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback'],
      dtype='object')

In [16]:
#Train-test split
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1,test_size=0.2)

In [17]:
# Define the model. Set random_state to 1
rf_model1 = RandomForestClassifier(n_estimators = 100, random_state=1)
rf_model2 = RandomForestClassifier(n_estimators = 50, random_state=2)
rf_model3 = RandomForestClassifier(n_estimators = 150, random_state=3)
rf_model4 = RandomForestClassifier(n_estimators = 200, random_state=4)
rf_model5 = RandomForestClassifier(n_estimators = 250, random_state=5)
# fit your model
rf_model1.fit(train_X,train_y)
val_preds1 = rf_model1.predict(val_X)
rf_model2.fit(train_X,train_y)
val_preds2 = rf_model2.predict(val_X)
rf_model3.fit(train_X,train_y)
val_preds3 = rf_model3.predict(val_X)
rf_model4.fit(train_X,train_y)
val_preds4 = rf_model4.predict(val_X)
rf_model5.fit(train_X,train_y)
val_preds5 = rf_model5.predict(val_X)

# using metrics module for accuracy calculation
print("VALIDATION ACCURACY OF THE MODEL 1: ", metrics.accuracy_score(val_y, val_preds1))
print("VALIDATION ACCURACY OF THE MODEL 2: ", metrics.accuracy_score(val_y, val_preds2))
print("VALIDATION ACCURACY OF THE MODEL 3: ", metrics.accuracy_score(val_y, val_preds3))
print("VALIDATION ACCURACY OF THE MODEL 4: ", metrics.accuracy_score(val_y, val_preds4))
print("VALIDATION ACCURACY OF THE MODEL 5: ", metrics.accuracy_score(val_y, val_preds5))

VALIDATION ACCURACY OF THE MODEL 1:  0.7538856602243547
VALIDATION ACCURACY OF THE MODEL 2:  0.7478037572644952
VALIDATION ACCURACY OF THE MODEL 3:  0.7548317340181105
VALIDATION ACCURACY OF THE MODEL 4:  0.7526692796323827
VALIDATION ACCURACY OF THE MODEL 5:  0.7538856602243547


In [18]:
test_preds1 = rf_model1.predict(test_X)
test_preds2 = rf_model2.predict(test_X)
test_preds3 = rf_model3.predict(test_X)
test_preds4 = rf_model4.predict(test_X)
test_preds5 = rf_model5.predict(test_X)
test_preds = []
print(test_preds5[0])
for i in range(len(test_preds1)):
    lst = [test_preds1[i], test_preds2[i], test_preds3[i], test_preds4[i], test_preds5[i]]
    test_preds.append(max(lst,key=lst.count))

3


In [19]:
# The lines below shows how to save predictions in format used for competition scoring.
output = pd.DataFrame({'customer_id': test_data.customer_id,
                       'churn_risk_score': test_preds})

output.head()
output.to_csv('submission.csv', index=False)

In [20]:
print(list(test_imputed.columns))

['customer_id', 'Name', 'age', 'gender', 'security_no', 'region_category', 'membership_category', 'joining_date', 'joined_through_referral', 'referral_id', 'preferred_offer_types', 'medium_of_operation', 'internet_option', 'last_visit_time', 'days_since_last_login', 'avg_time_spent', 'avg_transaction_value', 'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount', 'offer_application_preference', 'past_complaint', 'complaint_status', 'feedback']


In [21]:
feature_imp = pd.Series(rf_model1.feature_importances_).sort_values(ascending = False)
feature_imp

3     0.269082
2     0.129585
1     0.086369
0     0.075534
10    0.065112
12    0.063519
15    0.028151
14    0.028086
13    0.026961
27    0.024958
25    0.024596
29    0.024213
26    0.023622
11    0.023493
17    0.008683
28    0.007936
7     0.007809
8     0.007795
22    0.007597
23    0.007492
21    0.007398
4     0.007249
5     0.007239
24    0.007003
19    0.006091
18    0.006080
9     0.006078
16    0.005976
20    0.005968
6     0.000326
dtype: float64